# Handle missing values

In [19]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Practice").getOrCreate()

In [20]:
df = spark.read.csv("base_salary.csv", header=True,inferSchema=True)

In [21]:
df.show()

+--------+----+----------+--------+
|    name| age|experience|salary  |
+--------+----+----------+--------+
|   Roger|24.0|         2|   24000|
|   Marie|40.0|        10|   40000|
|  Fatima|47.0|         7|   80000|
|    Alex|26.0|         3|   38000|
|   sunny|24.0|         2|    NULL|
|    NULL|25.0|      NULL|    NULL|
|  Robert|50.0|      NULL|    NULL|
|    NULL|48.0|      NULL|   60000|
|Amandine|NULL|         5|    NULL|
|    NULL|NULL|      NULL|    NULL|
|  Armine|NULL|      NULL|    NULL|
+--------+----+----------+--------+



In [22]:
df.describe().show()

+-------+-----+------------------+------------------+------------------+
|summary| name|               age|        experience|          salary  |
+-------+-----+------------------+------------------+------------------+
|  count|    8|                 8|                 6|                 5|
|   mean| NULL|              35.5| 4.833333333333333|           48400.0|
| stddev| NULL|11.856282240712245|3.1885210782848317|21835.750502329887|
|    min| Alex|              24.0|                 2|             24000|
|    max|sunny|              50.0|                10|             80000|
+-------+-----+------------------+------------------+------------------+



## Drop ine with null values

In [23]:
df.na.drop().show()

+------+----+----------+--------+
|  name| age|experience|salary  |
+------+----+----------+--------+
| Roger|24.0|         2|   24000|
| Marie|40.0|        10|   40000|
|Fatima|47.0|         7|   80000|
|  Alex|26.0|         3|   38000|
+------+----+----------+--------+



In [24]:
df.na.drop(how="all").show() #Any by default

+--------+----+----------+--------+
|    name| age|experience|salary  |
+--------+----+----------+--------+
|   Roger|24.0|         2|   24000|
|   Marie|40.0|        10|   40000|
|  Fatima|47.0|         7|   80000|
|    Alex|26.0|         3|   38000|
|   sunny|24.0|         2|    NULL|
|    NULL|25.0|      NULL|    NULL|
|  Robert|50.0|      NULL|    NULL|
|    NULL|48.0|      NULL|   60000|
|Amandine|NULL|         5|    NULL|
|  Armine|NULL|      NULL|    NULL|
+--------+----+----------+--------+



In [28]:
df.na.drop(thresh=2).show() # At least 2 NONE values

+--------+----+----------+--------+
|    name| age|experience|salary  |
+--------+----+----------+--------+
|   Roger|24.0|         2|   24000|
|   Marie|40.0|        10|   40000|
|  Fatima|47.0|         7|   80000|
|    Alex|26.0|         3|   38000|
|   sunny|24.0|         2|    NULL|
|  Robert|50.0|      NULL|    NULL|
|    NULL|48.0|      NULL|   60000|
|Amandine|NULL|         5|    NULL|
+--------+----+----------+--------+



## Selecting columns and indexing